In [ ]:
import sys
from collections import OrderedDict
from pyexcel_ods import get_data
from config_rm import *
from redminelib import Redmine

Ahora procedemos a cargar el fichero

In [ ]:
file_path = "input_wbs.ods"
data = get_data(file_path)['Export2Redmine']
tasks = {}
print("file loaded")

In [ ]:
cfg_prefix = "wp"
cfg_name_prefix = "WP"
cfg_tgt_prefix = "T"
cfg_first_valid_row = 3
cfg_import_column = 0
cfg_level_column = 4
cfg_ident_column = 1
cfg_name_column = 17
cfg_parent_column = 14
cfg_description_column = 18
cfg_assignee_column = 19
cfg_prefix_column = 20


In [ ]:
def processTgt(row, lastPrj, projects):
    level = int(row[cfg_level_column])
    thisTgt = {}
    print("tgt:",row[cfg_ident_column])
    thisTgt['level'] = level
    thisTgt['description'] = row[cfg_name_column]
    if (len(row) > cfg_description_column):
        thisTgt['description'] += "\n\n" + row[cfg_description_column]
    
    if (len(row[cfg_parent_column])>0):
        thisTgt['parent'] = row[cfg_parent_column]
        idx = projects[row[cfg_parent_column]]['tgtnum'] + 1
        thisTgt['ident'] = projects[row[cfg_parent_column]]['prefix'] + "."+cfg_tgt_prefix+"{:02d}".format(idx)
        projects[row[cfg_parent_column]]['tgtnum'] = idx
        projects[row[cfg_parent_column]]['targets'][row[cfg_ident_column]] = thisTgt

    else:
        thisTgt['parent'] = None
        idx = projects['root']['tgtnum'] + 1
        thisTgt['ident'] = cfg_tgt_prefix + "{:02d}".format(idx)
        projects['root']['tgtnum'] = idx
        projects['root']['targets'][row[cfg_ident_column]] = thisTgt
    
    return thisTgt    

def processPrj(row, lastPrj, projects):
    level = int(row[cfg_level_column])
    thisPrj = {}
    print("prj:",row[1])
    thisPrj['level'] = level
    thisPrj['description'] = row[cfg_name_column]
    thisPrj['childnum'] = 0
    thisPrj['tgtnum'] = 0
    thisPrj['targets'] = {}
    if (len(row) > cfg_description_column):
        thisPrj['description'] += "\n\n" + row[cfg_description_column]
    
    if (len(row[cfg_parent_column])>0):
        thisPrj['parent'] = row[cfg_parent_column]
        idx = projects[row[cfg_parent_column]]['childnum'] + 1
        projects[row[cfg_parent_column]]['childnum'] = idx
        thisPrj['name'] = projects[row[cfg_parent_column]]['name'] + ".{:02d}".format(idx)
        thisPrj['ident'] = projects[row[cfg_parent_column]]['ident'] + "-{:02d}".format(idx)

    else:
        thisPrj['parent'] = None
        idx = projects['root']['childnum'] + 1
        projects['root']['childnum'] = idx
        thisPrj['name'] = cfg_name_prefix + "{:02d}".format(idx)
        thisPrj['ident'] = cfg_prefix + "{:02d}".format(idx)
    
    if (len(row) > cfg_assignee_column):
        thisPrj['assignee'] = row[cfg_assignee_column]

    if (len(row) > cfg_assignee_column):
        thisPrj['prefix'] = row[cfg_prefix_column]
        
        
    return thisPrj

def process_row(row, lastPrj, projects):
    level = int(row[cfg_level_column])
    if (level > 0):
        lastPrj = processPrj(row,lastPrj,projects)
        projects[row[cfg_ident_column]] = lastPrj
    else:
        lastTgt = processTgt(row, lastPrj, projects)
        targets[row[cfg_ident_column]] = lastTgt
        
    return lastPrj

i = 0
projects = {}
projects['root'] = {}
projects['root']['childnum'] = 0
projects['root']['tgtnum'] = 0
projects['root']['targets'] = {}
lastPrj = None
targets = {}

for row in data:
    # Let's skip the first row
    if (i >= cfg_first_valid_row):
        if (len(row) > cfg_import_column):
            if (row[cfg_import_column]>0):
                #print("processing row",i)
                lastPrj = process_row(row,lastPrj,projects)

    i += 1
    
print("done")

In [ ]:
def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key))
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))
            
pretty(projects)

In [ ]:
pretty(targets)

In [ ]:
print(projects.keys())

In [ ]:
print(targets.keys())

Vamos a ver si ya existen los proyectos, y los vamos a anidar en la estructura que ya tenemos, o los crearemos nuevos.

In [ ]:
redmine = Redmine(rm_server_url,key=rm_key_txt)
rmprojects = redmine.project.all()

rmprojectsbyident = {}
print("Proyectos:")
for p in rmprojects:
    print ("\t",p.identifier," \t| ",p.name)
    rmprojectsbyident[p.identifier] = p

root_rmproject = redmine.project.get(rm_project_id_str)
print ("Obtenemos proyecto raíz: ",root_rmproject.identifier," | ",root_rmproject.name)

In [ ]:
for k in projects.keys():
    if (k != 'root'):
        thisproject = projects[k]
        prident = thisproject['ident']
        print(k,prident)
        print(thisproject)
        rmprj = None
        if (prident in rmprojectsbyident.keys()):
            print("Está en la lista, lo busco")
            rmprj = redmine.project.get(prident)
            if (rmprj is not None):
                print("Lo hemos encontrado")
                projects[k]['rmprj'] = rmprj
                
        if (thisproject['parent'] is None):
            print("No hay padre")
            rmprojectid = root_rmproject.id
            
        else:
            print("Sí hay padre")
            rmprojectid = projects[thisproject['parent']]['rmid']
        
        print("rmprojectid",rmprojectid,thisproject['ident'])
          
        if rmprj is None:
            rmprj = redmine.project.create(
                    name=thisproject['name'],
                    identifier=thisproject['ident'],
                    description=thisproject['description'][:250],
                    is_public=True,
                    parent_id=rmprojectid,
                    inherit_members=True
            )
            thisproject['rmprj'] = rmprj            
        else:
            redmine.project.update(
                    rmprj.id,
                    name=thisproject['name'],
                    identifier=thisproject['ident'],
                    description=thisproject['description'][:250],
                    is_public=True,
                    parent_id=rmprojectid,
                    inherit_members=True
            )
        
        thisproject['rmid'] = rmprj.id

        '''
        for v in rmprj.versions:
            redmine.version.delete(v.id)
        '''
        for k in thisproject['targets']:
            thistgt = targets[k]
            tgtident = thistgt['ident']
            print(k,tgtident)
            print(thistgt)
        
            rmtgt = None
            for v in rmprj.versions:
                print("version:",v.name,"?",thistgt['ident'])
                if v.name == thistgt['ident']:
                    print("lo encontré")
                    rmtgt = v

            if rmtgt is None:
                print("Creamos el target")
                rmtgt = redmine.version.create(
                     project_id=rmprj.id,
                     name=thistgt['ident'],
                     description=thistgt['description'][:250]
                )            

            else:
                print("Modificamos el target")
                redmine.version.update(
                    rmtgt.id,
                    name=thistgt['ident'],
                    description=thistgt['description'][:250]
                )            
 












In [ ]:
print("done")

Code for removing versions (disabled)

In [ ]:
'''rmprojects = redmine.project.all()

for k in rmprojects:
    print(k.name)
    for v in k.versions:
        print(v.name)
        redmine.version.delete(v.id)
'''


In [ ]:
rmusers = redmine.user.all()
users = {}
for u in rmusers:
    #print(u.login)
    users[u.login] = u
    
print(users)
    


In [ ]:
roles = redmine.role.all()
for r in roles:
    print(r.name)
    if (r.name == 'RmMngr'):
        mngrole = r
        
trackers = redmine.tracker.all()
for tr in trackers:
    print(tr.name)
    if (tr.name == 'wrk'):
        ko_tracker = tr
        dv_tracker = tr
        cl_tracker = tr
        

In [ ]:
for k in projects.keys():
    if (k!='root'):
        p = projects[k]
        print(p)
        print(p['rmid'])
        thispr = redmine.project.get(p['rmid'])
        print(thispr)        
        print(k,p['ident'],p['assignee'])
        if p['assignee'] not in users.keys():
            print("ERROR no existe el usuario",p['assignee'])
            
        else:
            u = users[p['assignee']]
            print("userid",u.id)
            created = False
            members = thispr.memberships
            for m in members:
                print(dict(m))
                print(m.user.id)
                print(m.roles)
                if (m.user.id == u.id):
                    print("El usuario está")
                    created = True
                    done = False
                    thisroles = []
                    for r in m.roles:
                        thisroles += [r.id]
                        if mngrole.id == r.id:
                            print("El rol también está")
                            done = True
                    
                    if not done:
                        thisroles += [mngrole.id]
                        print(thisroles)
                        m2 = redmine.project_membership.get(m.id)
                        redmine.project_membership.update(m.id, role_ids=[1, 2])
            
            if not created:           
                redmine.project_membership.create(project_id=p['rmid'], user_id=users[p['assignee']].id, role_ids=[mngrole.id])


In [ ]:
for k in projects.keys():
    if (k!='root'):
        issues = {}
        p = projects[k]['rmprj']
        u = users[projects[k]['assignee']]
        for i in p.issues:
            issues[i.subject] = i
            print(i.subject)
        
        for t in p.versions:
            print(k,t,p.identifier,u.firstname)
            # kickoff
            ko_subject = t.name + " kickoff"
            ko_descr = "Kick-Off meeting for deliverable named '"+t.name+"'"
            print("pruebo ",ko_subject,issues.keys())
            if ko_subject in issues.keys():
                print("estA!")
                ko_i = issues[ko_subject]
                redmine.issue.update(
                    ko_i.id,
                    tracker_id=ko_tracker.id,
                    description=ko_descr,
                    fixed_version_id=t.id,
                    assigned_to_id=u.id          
                )
            else:
                ko_i = redmine.issue.create(
                    project_id=p.id,
                    subject=ko_subject,
                    tracker_id=ko_tracker.id,
                    description=ko_descr,
                    fixed_version_id=t.id,
                    assigned_to_id=u.id
                )
                issues[ko_subject] = ko_i
            # devel
            ko_dv_rel_done = False
            dv_subject = t.name + " development"
            dv_descr = "Development for deliverable named '"+t.name+"'"
            print("pruebo ",dv_subject,issues.keys())            
            if dv_subject in issues.keys():
                print("estA!")
                dv_i = issues[dv_subject]
                redmine.issue.update(
                    dv_i.id,
                    tracker_id=dv_tracker.id,
                    description=dv_descr,
                    fixed_version_id=t.id,
                    assigned_to_id=u.id   
                )
                for r in dv_i.relations:
                    print(dict(r))
                    if r.issue_id == ko_i.id:
                        ko_dv_rel_done = True
            else:
                dv_i = redmine.issue.create(
                    project_id=p.id,
                    subject=dv_subject,
                    tracker_id=dv_tracker.id,
                    description=dv_descr,
                    fixed_version_id=t.id,
                    assigned_to_id=u.id
                )
                issues[dv_subject] = dv_i
                
            if not ko_dv_rel_done:
                # Create the relation
                ko_dv_rel = redmine.issue_relation.create(
                    issue_id=ko_i.id,
                    issue_to_id=dv_i.id,
                    relation_type='precedes',
                    delay=0
                )
            # devel
            dv_cl_rel_done = False            
            cl_subject = t.name + " closure"
            cl_descr = "Closure for deliverable named '"+t.name+"'"
            print("pruebo ",cl_subject,issues.keys())                      
            if cl_subject in issues.keys():
                print("estA!")
                cl_i = issues[cl_subject]
                redmine.issue.update(
                    cl_i.id,
                    tracker_id=cl_tracker.id,
                    description=cl_descr,
                    fixed_version_id=t.id,
                    assigned_to_id=u.id          
                )
                for r in cl_i.relations:
                    print(dict(r))
                    if r.issue_id == dv_i.id:
                        dv_cl_rel_done = True                
            else:
                cl_i = redmine.issue.create(
                    project_id=p.id,
                    subject=cl_subject,
                    tracker_id=cl_tracker.id,
                    description=cl_descr,
                    fixed_version_id=t.id,
                    assigned_to_id=u.id
                )
                issues[cl_subject] = cl_i
                
            if not dv_cl_rel_done:
                # Create the relation
                dv_cl_rel = redmine.issue_relation.create(
                    issue_id=dv_i.id,
                    issue_to_id=cl_i.id,
                    relation_type='precedes',
                    delay=0
                )

In [ ]:
print("done")

In [ ]:
print(issues)